In [232]:
import torch
from torch import nn
from torch.nn import functional as F
from torch import optim
import numpy as np
import pandas as pd
from rnn import StochasticLSTM
from utils import weight_coefficient, bias_coefficient, filter_parameters, train_model

In [233]:
from sklearn.metrics import roc_auc_score

In [234]:
training_table = pd.read_table("./data/hill-valley/Hill_Valley_with_noise_Training.data", sep=',', dtype=np.float64)
testing_table = pd.read_table("./data/hill-valley/Hill_Valley_with_noise_Testing.data", sep=',', dtype=np.float64)

input_columns = [f"X{i}" for i in range(1, 101)]
label_column = "class"

In [235]:
training_table.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X92,X93,X94,X95,X96,X97,X98,X99,X100,class
0,39.02,36.49,38.20,38.85,39.38,39.74,37.02,39.53,38.81,38.79,...,36.62,36.92,38.80,38.52,38.07,36.73,39.46,37.50,39.10,0.0
1,1.83,1.71,1.77,1.77,1.68,1.78,1.80,1.70,1.75,1.78,...,1.80,1.79,1.77,1.74,1.74,1.80,1.78,1.75,1.69,1.0
2,68177.69,66138.42,72981.88,74304.33,67549.66,69367.34,69169.41,73268.61,74465.84,72503.37,...,73438.88,71053.35,71112.62,74916.48,72571.58,66348.97,71063.72,67404.27,74920.24,1.0
3,44889.06,39191.86,40728.46,38576.36,45876.06,47034.00,46611.43,37668.32,40980.89,38466.15,...,42625.67,40684.20,46960.73,44546.80,45410.53,47139.44,43095.68,40888.34,39615.19,0.0
4,5.70,5.40,5.28,5.38,5.27,5.61,6.00,5.38,5.34,5.87,...,5.17,5.67,5.60,5.94,5.73,5.22,5.30,5.73,5.91,0.0


In [236]:
dim = 1
seq_length = 100
DROP = 0.5

In [237]:
training_table.drop("class", axis=1)

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X91,X92,X93,X94,X95,X96,X97,X98,X99,X100
0,39.02,36.49,38.20,38.85,39.38,39.74,37.02,39.53,38.81,38.79,...,37.57,36.62,36.92,38.80,38.52,38.07,36.73,39.46,37.50,39.10
1,1.83,1.71,1.77,1.77,1.68,1.78,1.80,1.70,1.75,1.78,...,1.71,1.80,1.79,1.77,1.74,1.74,1.80,1.78,1.75,1.69
2,68177.69,66138.42,72981.88,74304.33,67549.66,69367.34,69169.41,73268.61,74465.84,72503.37,...,69384.71,73438.88,71053.35,71112.62,74916.48,72571.58,66348.97,71063.72,67404.27,74920.24
3,44889.06,39191.86,40728.46,38576.36,45876.06,47034.00,46611.43,37668.32,40980.89,38466.15,...,47653.60,42625.67,40684.20,46960.73,44546.80,45410.53,47139.44,43095.68,40888.34,39615.19
4,5.70,5.40,5.28,5.38,5.27,5.61,6.00,5.38,5.34,5.87,...,5.52,5.17,5.67,5.60,5.94,5.73,5.22,5.30,5.73,5.91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601,2064.63,2174.44,2249.02,2249.60,2307.61,2328.84,2071.90,2107.05,2242.68,2183.03,...,2325.98,2202.95,2248.56,2185.22,2145.38,2288.64,2288.42,2096.08,2114.25,2281.91
602,9.42,9.30,9.66,9.63,9.15,9.91,9.45,9.84,9.36,9.15,...,9.19,9.85,9.76,9.82,9.64,9.40,9.53,9.23,9.82,9.57
603,5.31,5.03,5.17,5.02,5.20,5.16,5.11,5.27,5.19,5.13,...,5.08,5.19,5.06,5.28,5.28,5.19,5.27,5.14,5.12,5.04
604,1.03,1.02,1.09,1.06,1.03,1.03,0.98,0.97,1.13,0.97,...,1.11,1.11,1.06,0.96,1.06,1.01,1.07,1.10,1.11,1.07


In [238]:
def scale_inputs(x_inputs):
    median = np.median(x_inputs, axis=1).reshape(-1, 1)
    shifted_x = x_inputs - median
    scale = np.abs(shifted_x).max(axis=1).reshape(-1, 1)
    return shifted_x / scale

In [239]:
x_train = scale_inputs(training_table.drop("class", axis=1).values)
y_train = training_table["class"].values
x_test = scale_inputs(testing_table.drop("class", axis=1).values)
y_test = testing_table["class"].values

N = len(y_train)

train_dl = torch.utils.data.DataLoader(
        [(x_train[i], y_train[i]) for i in range(len(y_train))],
        batch_size=10,
        num_workers=2,
        shuffle=True
)

def format_input(input_batch):
    return input_batch.transpose(1, 0).reshape(seq_length, -1, dim)

In [240]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.rnn = StochasticLSTM(1, 10, DROP)
        self.fc = nn.Linear(10, 1)
    
    def forward(self, x):
        out, _ = self.rnn(x)
        out = torch.tanh(out[-1])
        out = self.fc(out)
        return out

In [241]:
net = Net().double()
criterion = nn.BCEWithLogitsLoss()

M = filter_parameters(net.named_parameters(), "rnn", "weight")
m = filter_parameters(net.named_parameters(), "rnn", "bias")
other_params = filter_parameters(net.named_parameters(), "fc", "")

parameters = [
    {"params": M, "weight_decay": weight_coefficient(1, 1, DROP, N)}, # dropout rnn weight
    {"params": m, "weight_decay": bias_coefficient(1, 1, N)}, # dropout rnn bias
    {"params": other_params} # other parameters
]
optimizer = optim.Adam(parameters, lr=1e-3)

In [242]:
net.train()
for epoch in range(70):
    running_loss = 0.0
    
    for i, (inputs, labels) in enumerate(train_dl):
        inputs = format_input(inputs)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs.flatten(), labels.double())
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 60 == 59:
            print('[%d, %5d] loss: %.6f' %
                  (epoch + 1, i + 1, running_loss / 60))
            running_loss = 0.0

net.eval()
print("Finish training")

[1,    60] loss: 0.691331
[2,    60] loss: 0.687976
[3,    60] loss: 0.684752
[4,    60] loss: 0.677401
[5,    60] loss: 0.665819
[6,    60] loss: 0.632883
[7,    60] loss: 0.596804
[8,    60] loss: 0.572507
[9,    60] loss: 0.548868
[10,    60] loss: 0.549237
[11,    60] loss: 0.522626
[12,    60] loss: 0.505411
[13,    60] loss: 0.478787
[14,    60] loss: 0.479767
[15,    60] loss: 0.474467
[16,    60] loss: 0.451829
[17,    60] loss: 0.469593
[18,    60] loss: 0.466033
[19,    60] loss: 0.462533
[20,    60] loss: 0.456150
[21,    60] loss: 0.450837
[22,    60] loss: 0.439042
[23,    60] loss: 0.446072
[24,    60] loss: 0.448176
[25,    60] loss: 0.453790
[26,    60] loss: 0.425476
[27,    60] loss: 0.430693
[28,    60] loss: 0.429815
[29,    60] loss: 0.422505
[30,    60] loss: 0.430798
[31,    60] loss: 0.428890
[32,    60] loss: 0.412452
[33,    60] loss: 0.407932
[34,    60] loss: 0.411501
[35,    60] loss: 0.413657
[36,    60] loss: 0.398388
[37,    60] loss: 0.400976
[38,    60

In [243]:
with torch.no_grad():
    outputs = net(torch.tensor(x_test.reshape(606, 100, 1).swapaxes(1, 0)))
    outputs = outputs.flatten().data.numpy()

In [244]:
roc_auc_score(y_test, outputs)

0.9275434946019849